In [94]:
from tavily import TavilyClient
from langchain_core.tools import tool
import os
from typing import Literal
from agentic_blocks.utils.tools_utils import create_tool_registry, execute_and_add_tool_responses
from agentic_blocks.utils.tools_utils import langchain_tool_to_openai_format
from agentic_blocks import call_llm, Messages

In [95]:
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [96]:
@tool
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search"""
    search_docs = tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )
    return search_docs

tool_registry = create_tool_registry([internet_search])

In [97]:
langchain_tool_to_openai_format(internet_search)

{'type': 'function', 'function': {'name': 'internet_search', 'description': 'Run a web search', 'parameters': {'type': 'object', 'properties': {'query': {'title': 'Query', 'type': 'string'}, 'max_results': {'default': 5, 'title': 'Max Results', 'type': 'integer'}, 'topic': {'default': 'general', 'enum': ['general', 'news', 'finance'], 'title': 'Topic', 'type': 'string'}, 'include_raw_content': {'default': False, 'title': 'Include Raw Content', 'type': 'boolean'}}, 'required': ['query']}}}

In [ ]:
research_instructions = """You are an expert researcher. Your job is to conduct thorough research, and then write a polished report.

Use the internet_search tool to conduct deep research. It will respond to your questions/topics with a detailed answer.

When you think you enough information to write a final report provide the final report.

Here are instructions for writing the final report:

<report_instructions>

CRITICAL: If you make a todo plan - you should note in the plan what language the report should be in so you dont forget!

Note: Write the report in English.

Please create a detailed answer to the overall research brief that:
1. Is well-organized with proper headings (# for title, ## for sections, ### for subsections)
2. Includes specific facts and insights from the research
3. References relevant sources using [Title](URL) format
4. Provides a balanced, thorough analysis. Be as comprehensive as possible, and include all information that is relevant to the overall research question. People are using you for deep research and will expect detailed, comprehensive answers.
5. Includes a "Sources" section at the end with all referenced links

You can structure your report in a number of different ways. Here are some examples:

To answer a question that asks you to compare two things, you might structure your report like this:
1/ intro
2/ overview of topic A
3/ overview of topic B
4/ comparison between A and B
5/ conclusion

To answer a question that asks you to return a list of things, you might only need a single section which is the entire list.
1/ list of things or table of things
Or, you could choose to make each item in the list a separate section in the report. When asked for lists, you don't need an introduction or conclusion.
1/ item 1
2/ item 2
3/ item 3

To answer a question that asks you to summarize a topic, give a report, or give an overview, you might structure your report like this:
1/ overview of topic
2/ concept 1
3/ concept 2
4/ concept 3
5/ conclusion

If you think you can answer the question with a single section, you can do that too!
1/ answer

REMEMBER: Section is a VERY fluid and loose concept. You can structure your report however you think is best, including in ways that are not listed above!
Make sure that your sections are cohesive, and make sense for the reader.

For each section of the report, do the following:
- Use simple, clear language
- Use ## for section title (Markdown format) for each section of the report
- Do NOT ever refer to yourself as the writer of the report. This should be a professional report without any self-referential language. 
- Do not say what you are doing in the report. Just write the report without any commentary from yourself.
- Each section should be as long as necessary to deeply answer the question with the information you have gathered. It is expected that sections will be fairly long and verbose. You are writing a deep research report, and users will expect a thorough answer.
- Use bullet points to list out information when appropriate, but by default, write in paragraph form.

REMEMBER:
The brief and research may be in English, but you need to translate this information to the right language when writing the final answer.
Make sure the final answer report is in English.

Format the report in clear markdown with proper structure and include source references where appropriate.

<Citation Rules>
- Assign each unique URL a single citation number in your text
- End with ### Sources that lists each source with corresponding numbers
- IMPORTANT: Number sources sequentially without gaps (1,2,3,4...) in the final list regardless of which sources you choose
- Each source should be a separate line item in a list, so that in markdown it is rendered as a list.
- Example format:
  [1] Source Title: URL
  [2] Source Title: URL
- Citations are extremely important. Make sure to include these, and pay a lot of attention to getting these right. Users will often use these citations to look into more information.
</Citation Rules>
</report_instructions>

You have access to this tool:

## `internet_search`

Use this to run an internet search for a given query. You can specify the number of results, the topic, and whether raw content should be included.
"""

In [104]:
messages = Messages(
    system_prompt=research_instructions,
    user_prompt="Make a summary of the best restaurants in Stockholm"
)

model = "qwen/qwen3-235b-a22b-2507"
#model = "openai/gpt-oss-120b"

In [105]:
has_answer = False

while not has_answer:
    response = call_llm(model=model, messages=messages, tools=[internet_search])
    messages.add_response_message(response)
    #print(messages.get_messages()[-1])
    
    if messages.has_pending_tool_calls():
        tool_call = messages.get_pending_tool_calls()[0]
        print(f"Tool call: {tool_call['tool_name']}: {tool_call['arguments']['query']}")
        execute_and_add_tool_responses(messages, tool_registry)
    else:
        has_answer = True
    
    

Tool call: internet_search: best restaurants in Stockholm
Tool call: internet_search: Michelin-starred restaurants in Stockholm
Tool call: internet_search: Frantzén Stockholm restaurant menu and experience
Tool call: internet_search: Sushi Sho Stockholm review and experience
Tool call: internet_search: best traditional Swedish food restaurants Stockholm
Tool call: internet_search: Östermalms Saluhall Stockholm food market guide


In [106]:
print(response.content)

# Summary of the Best Restaurants in Stockholm

Stockholm offers a dynamic and diverse dining scene that ranges from world-renowned fine dining experiences to intimate traditional eateries and bustling food halls. The city has gained global recognition for its innovative approach to Nordic cuisine, especially highlighted by its Michelin-starred restaurants, while still preserving classic Swedish dishes in longstanding institutions. This guide summarizes some of the best and most celebrated restaurants in Stockholm, categorized by dining experience and culinary focus.

## Michelin-Starred Fine Dining Establishments

Stockholm is home to a total of 12 Michelin-starred restaurants, including one with the prestigious three-star rating—Frantzén—which places it among the elite culinary destinations in Europe. These restaurants emphasize meticulous preparation, seasonal ingredients, and creative fusion, often blending Swedish traditions with Japanese and modern gastronomic techniques.

### Fr